In [ ]:
import pandas as pd
import csv

product = "Kühlschränke" # e.g. "Waschmaschinen" / "Washing machines" or "Kühlschränke" / "Refrigerators"
only_first_timepoint = False # Set to True if only the first timepoint should be analyzed

def detect_delimiter(filename):
    with open(filename, 'r') as file:
        sniffer = csv.Sniffer()
        return sniffer.sniff(file.read(5000)).delimiter

# To adjust for different column names in the CSV files
column_mapping = {
    'Model': ['Model', 'Product', 'Item'],
    'Vendor': ['Vendor', 'Seller', 'Retailer'],
    'Price_w/o_shipping': ['Price_w/o_shipping', 'Price', 'Cost'],
    'Date': ['Date', 'Timestamp', 'SaleDate']
}

filename = f"{product}/{product}.csv"
delimiter = detect_delimiter(filename)

df = pd.read_csv(filename, sep=delimiter, dtype=str)

# Standardize column names
df.columns = df.columns.str.strip().str.lower()
for standard_name, possible_names in column_mapping.items():
    for possible_name in possible_names:
        if possible_name.lower() in df.columns:
            df.rename(columns={possible_name.lower(): standard_name}, inplace=True)
            break

display(df)

In [ ]:
# Convert Price column correct format. If the price is already in correct format, this will have no effect
df["Price_w/o_shipping"] = (
    df["Price_w/o_shipping"]
    .str.replace(r'[^\d.,]', '', regex=True)  # Remove any non-numeric characters except commas and periods
    .str.replace(",", ".")                    # Replace comma with period
    .astype(float)
)

# Create column for manufacturer by taking first word of Model name (only if there is no Manufacturer column already in the dataset)
if 'Manufacturer' not in df.columns:
    df["Manufacturer"] = df["Model"].str.split().str[0]

# Drop all columns except the ones we need
df = df[['Model','Price_w/o_shipping','Vendor', 'Manufacturer', 'Date']]
print(f"Initial number of rows: {len(df)}")

# Drop rows with NA
df = df.dropna()
print(f"After dropping NA: {len(df)}")

display(df)

In [ ]:
if only_first_timepoint:    
    df['Date'] = pd.to_datetime(df['Date'], format='%d.%m.%Y %H:%M')
    #earliest_timestamp = df['Date'].min()
    target_date = pd.to_datetime('09.11.2022 18:00', format='%d.%m.%Y %H:%M') # change depending on dataset
    df = df[df['Date'] == target_date]

    display(df)

In [ ]:
# Delete rows where Manufacturer name is in Vendor (meaning that the vendor is a store operated by the manufacturer). No RPM possible when vendor and manufacturer is the same company
df['Manufacturer'] = df['Manufacturer'].astype(str)
df['Vendor'] = df['Vendor'].astype(str)
df = df[~df.apply(lambda row: row['Manufacturer'] in row['Vendor'], axis=1)]
print(f"After dropping offers by the manufacturer themselves: {len(df)}")

# Convert all vendors to lowercase, to avoid one vendor being counted multiple times because of capitalization differences
df['Vendor'] = df['Vendor'].apply(lambda x: x.lower())

# Drop duplicates
df = df.drop_duplicates().reset_index(drop=True)
print(f"After dropping duplicates: {len(df)}")

# Exclude rows where the same vendor offers the same model at the same date for different prices (there may be variants of the model, like different colors). Keep the lowest price
idx = df.groupby(['Model', 'Vendor', 'Date'])['Price_w/o_shipping'].idxmin()
df = df.loc[idx].reset_index(drop=True)
print(f"After dropping variants of each model: {len(df)}")

# Calculate the number of vendors for each product
df["Count_vendors"] = df.groupby(["Model", "Date"])["Model"].transform("count")

# Add median, std, mean and coef_var per distinct model and date pair.
df_grouped = df.groupby(['Model', 'Date']).agg(Median=('Price_w/o_shipping', 'median'), Std=('Price_w/o_shipping', 'std'), Mean=('Price_w/o_shipping', 'mean')).reset_index()
df = df.merge(df_grouped, on=["Model","Date"])

# Add Price_ratio (ratio of the price of the offering compared to the median & mean price)
df['Price_ratio_median'] = df['Price_w/o_shipping'] / df['Median']
df['Price_ratio_mean'] = df['Price_w/o_shipping'] / df['Mean']

# Remove outliers (50% above median, 50% below median) --> if there are many timepoints in the dataset, this will remove a lot of data, as a typo by the vendor is removed multiple times
df = df[df['Price_ratio_median'] <= 1.5]
print(f"After removing outliers double the median price: {len(df)}")
df = df[df['Price_ratio_median'] > 0.5]
print(f"After removing outliers half the median price: {len(df)}")

# Print counts of distinct vendors, models and manufacturers remaining in the dataset
num_vendors = df['Vendor'].nunique()
num_models = df['Model'].nunique()
num_hersteller = df['Manufacturer'].nunique()
print(f"Before Amthauer filtering, the dataset had {len(df)} rows, {num_vendors} vendors, {num_models} models from {num_hersteller} manufacturers")

# ------------------------------------------------------------- Default settings of Amthauer et al. (2023) (excl. 3 Model rule) -----------------------------------------------------------------------------------

# "We only include a model at one point in time if at this time it was offered by at least 5 vendors, to ensure that variation is generally possible." ------------------------------------------------------------
df = df[df['Count_vendors'] >= 5]
print(f"After 5 Vendor filtering, the dataset had {len(df)} rows")

# Remove models, which are not offered at at least 75% of all distinct timepoints (only if not too much data is removed) ------------------------------------------------------------------------------------------

# Calculate the total number of unique dates
total_unique_dates = df['Date'].nunique()

# Calculate 75% of the total number of unique dates
threshold_75 = 0.75 * total_unique_dates

# Group by model Model and count unique dates for each model
model_date_counts = df.groupby('Model')['Date'].nunique()

# Remove models that appear in less than 75% of all distinct dates
below75_models = model_date_counts[model_date_counts < threshold_75].index.tolist()

# If user uploads data spanning multiple product categories scraped on different dates, this would remove most data. Hence, this checks if much data would be removed. If yes, skip this step.
if len(below75_models) * 0.25 < len(df):
    df = df[~df['Model'].isin(below75_models)]
    print(f"After 75% timepoints filtering, the dataset had {len(df)} rows ----------------------")
# -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Calculate the variance per model and timepoint pair across vendors. Calculate coef-var for each Model & Date combination
vendor_var = df.groupby(['Model', 'Date'])['Price_w/o_shipping'].agg(['mean', 'std', 'nunique']).reset_index()
vendor_var['coef_var_among_vendors'] = vendor_var['std'] / vendor_var['mean']
vendor_var.rename(columns={"mean": "mean_among_vendors", 'std': 'std_among_vendors', 'nunique': 'distinct_vendor_prices'}, inplace=True)
df = df.merge(vendor_var, on=["Model","Date"])

# Drop rows with NA (just in case any NA were created during the previous steps)
df = df.dropna()
print(f"After dropping NA: {len(df)}")

# Reorder columns
new_column_order = ['Model',
                    'Price_w/o_shipping',
                    'Vendor',
                    'Count_vendors',
                    'Manufacturer',
                    'Date',
                    'Median',
                    'Mean',
                    'Std',
                    'Price_ratio_median',
                    'Price_ratio_mean',
                    'distinct_vendor_prices',
                    'mean_among_vendors',
                    'std_among_vendors',
                    'coef_var_among_vendors']

df = df.reindex(columns=new_column_order)

# Get counts of distinct vendors, models and manufacturers again, to compare the numbers before and after cleaning
num_vendors = df['Vendor'].nunique()
num_models = df['Model'].nunique()
num_hersteller = df['Manufacturer'].nunique()

print(f"The dataset now has {len(df)} rows, {num_vendors} vendors, {num_models} models from {num_hersteller} manufacturers")
display(df)

In [ ]:
# Save the data
if only_first_timepoint:
    df.to_csv(f"{product}/{product}_first_timepoint_clean.csv")
else:
    df.to_csv(f"{product}/{product}_clean.csv")